## **1. Research Questions introduction**



	“Quais são os fatores (género, número de ratings, e popularidade) que mais influenciam a média de avaliação dos filmes no MovieLens?”

Isto permite:
	•	Explorar a estrutura e a distribuição dos dados (EDA);
	•	Criar métricas derivadas (popularidade, número de votos, média ponderada);
	•	Usar modelos simples (regressão linear / árvore de decisão) para quantificar o impacto de cada variável;
	•	E escalar a análise — localmente em DuckDB, e facilmente replicável em cloud (Athena/BigQuery).

Notas da reunião com o prof, 06.11
- Partilhar dificuldades com os colegas, documentar as escolhas no notebook 
- fazer um threshold para numero de ratings por exemplo que queremos considerar. abaixo de 100 pode ser pouco!
- sugeriu fazer uma análise temporal face a eventos da década por exemplo - 11 set torres gémeas , aparecimento da internet em 95
- existem users muito ativos, que contribuem com muitos comentários
- analisar de o género de filmes ou filmes mais populares vão variando ao longo do tempo. podemos ver se ha filmes mt antigos com ratings / tags recentes  - exemplo do sozinho em casa no natal??

Creio que podemos orientar cada um dos exploratory data analysis ao que queremos... depois mudar os titulos! nao sabia muito bem como por

## **2. Pipeline**

### **2.1. Import necessary libraries and packages**

In [ ]:
import polars as pl
import pandas as pd
import duckdb
import sys
import os
import gdown # use pip install gdown for this - to download files from google drive 


pip install needed:

pip install polars pyarrow duckdb gdown

In [ ]:
print(sys.executable)

#"C:\Users\SaraEstevesHenriques\AppData\Local\Programs\Python\Python313\python.exe" -m pip install pyarrow
# use version 313 of python

c:\Users\SaraEstevesHenriques\AppData\Local\Programs\Python\Python313\python.exe


### **2.2. Import dataset files - 32 stored on Google Drive**

The files that compose the dataset for a complete analysis were stored on a Google Drive folder accessible by the direct URL.
Source of files: https://grouplens.org/datasets/movielens/
Permalink: https://grouplens.org/datasets/movielens/32m/
Version: MovieLens 32M

Description from the source
MovieLens 32M movie ratings. Stable benchmark dataset. 32 million ratings and two million tag applications applied to 87,585 movies by 200,948 users. Collected 10/2023 Released 05/2024


In [57]:
# Google Drive folder URL
folder_url = "https://drive.google.com/drive/folders/1azViEYTwDAw6rsQoWKGWyw8iDxQiHWrs?usp=drive_link"
output_dir = "data"

os.makedirs(output_dir, exist_ok=True)

# Download files in the folder, where the dataset with 32m registries is stored in parquet format
gdown.download_folder(url=folder_url, output=output_dir, quiet=False, use_cookies=False)


Retrieving folder contents


Processing file 1Kb6G1Wjiz1TycoN8yS7l8a1HJCdGdqpe links_32m.parquet
Processing file 15iPCCprMP4AIUczoIQedLAGGEDgxnIMy movies_32m.parquet
Processing file 1-BUqGS_AxvUZcYZymNzMAk9l8JVsy1fs ratings_32m.parquet
Processing file 1H15eh-AlA-YS-5-IZbby-YWf5-5_LQ9F tags_32m.parquet


Retrieving folder contents completed
Building directory structure
Building directory structure completed
Downloading...
From: https://drive.google.com/uc?id=1Kb6G1Wjiz1TycoN8yS7l8a1HJCdGdqpe
To: c:\Users\SaraEstevesHenriques\OneDrive - VortalSA\Documents\GitHub\BDF25_7\BDF25_7_data\big_data\ml-32m\data\links_32m.parquet
100%|██████████| 1.34M/1.34M [00:00<00:00, 10.2MB/s]
Downloading...
From: https://drive.google.com/uc?id=15iPCCprMP4AIUczoIQedLAGGEDgxnIMy
To: c:\Users\SaraEstevesHenriques\OneDrive - VortalSA\Documents\GitHub\BDF25_7\BDF25_7_data\big_data\ml-32m\data\movies_32m.parquet
100%|██████████| 2.22M/2.22M [00:00<00:00, 11.7MB/s]
Downloading...
From (original): https://drive.google.com/uc?id=1-BUqGS_AxvUZcYZymNzMAk9l8JVsy1fs
From (redirected): https://drive.google.com/uc?id=1-BUqGS_AxvUZcYZymNzMAk9l8JVsy1fs&confirm=t&uuid=35c0c824-2f2b-4d76-a278-7d5850559914
To: c:\Users\SaraEstevesHenriques\OneDrive - VortalSA\Documents\GitHub\BDF25_7\BDF25_7_data\big_data\ml-32m\data\ratings_

['data\\links_32m.parquet',
 'data\\movies_32m.parquet',
 'data\\ratings_32m.parquet',
 'data\\tags_32m.parquet']

Note that on the first approach and exploration DuckBD was used locally and also on this repository. However, we found much simplier to have the connection and create the tables to work later.

In [5]:
#check the available parquet files
files = [f for f in os.listdir("data") if f.endswith(".parquet")]
print("Available Parquet files:")
for f in files:
    print(f)

Available Parquet files:
links_32m.parquet
movies_32m.parquet
ratings_32m.parquet
tags_32m.parquet


#### **2.2.1. Create Movies table**

In [46]:
con = duckdb.connect()

df_movies = pl.read_parquet("data/ratings_32m.parquet")
df_movies.head()

userId,movieId,rating,timestamp
i64,i64,f64,i64
1,17,4.0,944249077
1,25,1.0,944250228
1,29,2.0,943230976
1,30,5.0,944249077
1,32,5.0,943228858


In [56]:
#Show how to query here:

# Connect to DuckDB (in-memory or to a file)
con = duckdb.connect(database=":memory:")

# Define file path
movies_path = "data/movies_32m.parquet"

# Write your SQL
querymovies = f"""
SELECT title, genres
FROM read_parquet('{movies_path}')
WHERE genres LIKE '%Action%'
LIMIT 5
"""

# 🔹 Fetch as a Polars DataFrame
df_pl_query_movies = con.execute(querymovies).pl()

print(type(df_pl_query_movies))
print(df_pl_query_movies.head())

<class 'polars.dataframe.frame.DataFrame'>
shape: (5, 2)
┌─────────────────────────┬─────────────────────────────────┐
│ title                   ┆ genres                          │
│ ---                     ┆ ---                             │
│ str                     ┆ str                             │
╞═════════════════════════╪═════════════════════════════════╡
│ Heat (1995)             ┆ Action|Crime|Thriller           │
│ Sudden Death (1995)     ┆ Action                          │
│ GoldenEye (1995)        ┆ Action|Adventure|Thriller       │
│ Cutthroat Island (1995) ┆ Action|Adventure|Romance        │
│ Money Train (1995)      ┆ Action|Comedy|Crime|Drama|Thri… │
└─────────────────────────┴─────────────────────────────────┘


#### **2.2.2. Create Ratings table**

In [45]:
con = duckdb.connect()

df_ratings = pl.read_parquet("data/ratings_32m.parquet")
df_ratings.head()

userId,movieId,rating,timestamp
i64,i64,f64,i64
1,17,4.0,944249077
1,25,1.0,944250228
1,29,2.0,943230976
1,30,5.0,944249077
1,32,5.0,943228858


#### **2.2.3. Create Links table**

In [ ]:
con = duckdb.connect()


df_links = pl.read_parquet("data/links_32m.parquet")
df_links.head()

,movieId,imdbId,tmdbId
0,1,0114709,862
1,2,0113497,8844
2,3,0113228,15602
3,4,0114885,31357
4,5,0113041,11862


#### **2.3.4. Create Tags table**

In [44]:
con = duckdb.connect()

df_tags = pl.read_parquet("data/tags_32m.parquet")
df_tags.head()

userId,movieId,tag,timestamp
i64,i64,str,i64
22,26479,"""Kevin Kline""",1583038886
22,79592,"""misogyny""",1581476297
22,247150,"""acrophobia""",1622483469
34,2174,"""music""",1249808064
34,2174,"""weird""",1249808102


## **3. Exploratory Data Analysis 1**

### **3.0. Import parquet - small ratings version!! - DELETE AFTER**

The following dataframe was obtained by making a query in AWS with the goal of extract aggregated data about ratings, tags and number of users that rate the movies.
As the ratings database is very large, in this version it was used the 100k version of the file.

CREATE OR REPLACE TABLE moviedetails AS
SELECT m.movieId, m.title, m.genres, AVG(r.rating) AS average_rating, COUNT(r.rating) AS rating_count, COUNT(t."movieId") as tag_count_per_movie
FROM movies m
INNER JOIN ratings r on m."movieId" = r."movieId"
INNER join tags t on m."movieId" = t."movieId"
GROUP BY m.movieId, m.title, m.genres;

In [21]:

print ("This is the file directory:", os.getcwd())
parquet_path = os.path.join(os.getcwd(), "moviedetails.parquet")

print("Python executable:", sys.executable)
print("Parquet exists:", os.path.exists(parquet_path))


try:
    df_small = pl.read_parquet(parquet_path)
    print("Eager read shape:", df_small.shape)
    display(df_small.head())
except Exception as e:
    print("Eager read failed:", e)

This is the file directory: c:\Users\SaraEstevesHenriques\OneDrive - VortalSA\Documents\GitHub\BDF25_7\BDF25_7_data\big_data\ml-32m
Python executable: c:\Users\SaraEstevesHenriques\AppData\Local\Programs\Python\Python313\python.exe
Parquet exists: True
Eager read shape: (9255, 6)


movieId,title,genres,average_rating,rating_count,tag_count_per_movie
i64,str,str,f64,i64,i64
161582,"""Hell or High Water (2016)""","""Crime|Drama""",3.5625,3176,3176
1982,"""Halloween (1978)""","""Horror""",3.722222,8586,8586
52245,"""Blades of Glory (2007)""","""Comedy|Romance""",3.088235,2482,2482
62,"""Mr. Holland's Opus (1995)""","""Drama""",3.70625,6800,6800
508,"""Philadelphia (1993)""","""Drama""",3.613636,27324,27324


### **3.1. MovieDetails and Statistis Import cvs - results obtained and exported from query in aws**

In order to perform the analysis indicated previously but with the 32m registers from ratings database, it was necessary to perform the query recurring to AWS Athena.
The results of the query were downloaded to CSV, added on this repository / project uder the name "moviedetails.csv".
The following code transforms the results of the CSV into a dataframe.

The files were saved on S3, while the following query was executed on Athena:


**SELECT movies.movieId, movies.title, movies.genres, AVG(ratings.rating) AS average_rating, COUNT(ratings.rating) AS rating_count, COUNT(tag."movieId") as tag_count_per_movie, COUNT(DISTINCT ratings."userId") as user_count_per_movie
FROM movies
left join ratings on movies."movieId" = ratings."movieId"
left join tag on movies."movieId" = tag."movieId"
GROUP BY movies.movieId, movies.title, movies.genres**


This query was very heavy to run locally, after 20 minutes it was decided to use powerfull methods to run it.
The joins made, mainly to the ratings table are the reason for it.

This, dataframe, allows us to have an average rating per movie and the count of ratings made per each. Also the count of tags per movie and the number of different users that have evaluated it. 


In [22]:
#the cvs is on this folder and its called moviedetails csv
df_moviedetails = pl.read_csv("moviedetails.csv")
print("CSV read successfully. Shape:", df_moviedetails.shape)
display(df_moviedetails .head())


CSV read successfully. Shape: (87585, 7)


movieId,title,genres,average_rating,rating_count,tag_count_per_movie,user_count_per_movie
i64,str,str,f64,i64,i64,i64
20,"""Money Train (1995)""","""Action|Comedy|Crime|Drama|Thri…",2.871458,589922,589922,4306
171,"""Jeffrey (1995)""","""Comedy|Drama""",3.593487,27132,27132,1428
315,"""Specialist, The (1994)""","""Action|Drama|Thriller""",2.893674,102717,102717,11413
331,"""Tom & Viv (1994)""","""Drama""",3.289855,1656,1656,552
386,"""S.F.W. (1994)""","""Drama""",2.822281,5655,5655,377


#### **3.1.1. Data Preparation for MovieDetails**

2.	Calcular:

•nº total de utilizadores, filmes e avaliações - não sei se o número de users é preciso
•distribuição de rating (média, mediana, desvio padrão)
•nº de filmes por género
•nº médio de ratings por filme e por utilizador

3.	Visualizar:
	
•histograma das classificações
•top 10 géneros mais avaliados
•relação entre nº de ratings e média por filme


###### **3.1.1.1. Titles analysis**

The titles in most cases have the actual title and the movie year. The first step is to split both of this informations. 

In [23]:
df_moviedetails.head(20)

movieId,title,genres,average_rating,rating_count,tag_count_per_movie,user_count_per_movie
i64,str,str,f64,i64,i64,i64
20,"""Money Train (1995)""","""Action|Comedy|Crime|Drama|Thri…",2.871458,589922,589922,4306
171,"""Jeffrey (1995)""","""Comedy|Drama""",3.593487,27132,27132,1428
315,"""Specialist, The (1994)""","""Action|Drama|Thriller""",2.893674,102717,102717,11413
331,"""Tom & Viv (1994)""","""Drama""",3.289855,1656,1656,552
386,"""S.F.W. (1994)""","""Drama""",2.822281,5655,5655,377
…,…,…,…,…,…,…
1233,"""Boot, Das (Boat, The) (1981)""","""Action|Drama|War""",4.129334,14334228,14334228,15804
1545,"""Ponette (1996)""","""Drama""",3.849922,7716,7716,643
1772,"""Blues Brothers 2000 (1998)""","""Action|Comedy|Musical""",2.544862,135149,135149,3143


In [24]:
# Extract year and remove it from title
df_moviedetails = df_moviedetails.with_columns([
    # Extract year, the 4 numbers inside the "". If no year is found it will be null
    pl.col("title").str.extract(r'\((\d{4})\)', 1).cast(pl.Int64).alias("year"),
    # Remove year from title (removes the pattern " (YYYY)" or "(YYYY)")
    pl.col("title").str.replace(r'\s*\(\d{4}\)', '').str.strip_chars('"').alias("title")
])

In [25]:
df_moviedetails.head()

movieId,title,genres,average_rating,rating_count,tag_count_per_movie,user_count_per_movie,year
i64,str,str,f64,i64,i64,i64,i64
20,"""Money Train""","""Action|Comedy|Crime|Drama|Thri…",2.871458,589922,589922,4306,1995
171,"""Jeffrey""","""Comedy|Drama""",3.593487,27132,27132,1428,1995
315,"""Specialist, The""","""Action|Drama|Thriller""",2.893674,102717,102717,11413,1994
331,"""Tom & Viv""","""Drama""",3.289855,1656,1656,552,1994
386,"""S.F.W.""","""Drama""",2.822281,5655,5655,377,1994


The following line allows to conclude that not every movie has the correspondent year set.

In [26]:
df_moviedetails.null_count()
#12 missing values on the year

movieId,title,genres,average_rating,rating_count,tag_count_per_movie,user_count_per_movie,year
u32,u32,u32,u32,u32,u32,u32,u32
0,0,0,3153,0,0,0,615


In [27]:
df_moviedetails_noyear = df_moviedetails.filter(pl.col("year").is_null())
df_moviedetails_noyear.head ()

movieId,title,genres,average_rating,rating_count,tag_count_per_movie,user_count_per_movie,year
i64,str,str,f64,i64,i64,i64,i64
146796,"""Le strelle nel fosso""","""(no genres listed)""",null,0,1,0,null
156605,"""Paterson""","""(no genres listed)""",3.712108,177940,177940,1148,null
191403,"""New Neighbors, Old Fights""","""(no genres listed)""",3.5,1,0,1,null
195217,"""Star Wars: Dresca""","""Sci-Fi""",2.590909,88,88,11,null
205074,"""Falling Inn Love""","""Comedy|Romance""",2.7,735,735,105,null


For the purpose of our analysis i think we'll need to remove the 3153 movies without average rating, these are movies without any rating

In [28]:
df_moviedetails_norating = df_moviedetails.filter(pl.col("average_rating").is_null())
df_moviedetails_norating.head (10)

movieId,title,genres,average_rating,rating_count,tag_count_per_movie,user_count_per_movie,year
i64,str,str,f64,i64,i64,i64,i64
101237,"""9500 Liberty""","""Documentary""",null,0,4,0,2009
109647,"""Bengazi""","""Adventure|Crime|Drama""",null,0,4,0,1955
110822,"""Super Fly T.N.T.""","""Action|Crime|Drama""",null,0,1,0,1973
116594,"""Driftin' River""","""Western""",null,0,1,0,1946
121395,"""Mother is a Freshman""","""Comedy""",null,0,3,0,1949
123794,"""Killer by Night""","""Crime|Thriller""",null,0,3,0,1972
129988,"""Tiffany Memorandum""","""Action|Romance""",null,0,1,0,1967
133497,"""Goldface il fantastico Superma…","""Adventure|Crime""",null,0,1,0,1967
138658,"""I Put a Hit On You""","""Comedy|Romance|Thriller""",null,0,1,0,2014


##### **3.1.1.2. Genres treatment**

There are movies with multiple genres and others with no gender indicated "(no genres listed)". The first action here is to treat the no genres listed as nulls. One possible approach to split the genders is use explode funcion, creating a line per movie per gender.

In [29]:
#Tranform no (no genres list) to null values
df_moviedetails = df_moviedetails.with_columns(
    pl.when(pl.col("genres").str.to_lowercase().str.strip_chars() == "(no genres listed)")
    .then(None)
    .otherwise(pl.col("genres"))
    .alias("genres")
    )

In [30]:
df_moviedetails_nogenre = df_moviedetails.filter(pl.col("genres").is_null())
df_moviedetails_nogenre.head (10)

movieId,title,genres,average_rating,rating_count,tag_count_per_movie,user_count_per_movie,year
i64,str,str,f64,i64,i64,i64,i64
123989,"""Spike of Bensonhurst""",null,4.5,26,26,2,1988
128393,"""Highway Racer""",null,3.5,21,21,3,1977
131320,"""Fort Yuma""",null,3.5,1,1,1,1955
132896,"""Pelota""",null,3.5,2,2,2,1983
137140,"""Blue Ridge""",null,5.0,1,0,1,2012
139375,"""Après la bataille""",null,null,0,1,0,2014
142128,"""The Missing Corpse""",null,1.5,1,0,1,1945
143217,"""Middle Age Crazy""",null,3.5,2,2,2,1980
144998,"""Rebels in Canada""",null,null,0,1,0,1965


In [31]:

df_moviedetails_nogenre.null_count()


movieId,title,genres,average_rating,rating_count,tag_count_per_movie,user_count_per_movie,year
u32,u32,u32,u32,u32,u32,u32,u32
0,0,7080,525,0,0,0,373


In [32]:
df_moviedetails_exploded = df_moviedetails.with_columns(
    pl.col("genres").str.split("|")
).explode("genres")

df_moviedetails_exploded.head()

movieId,title,genres,average_rating,rating_count,tag_count_per_movie,user_count_per_movie,year
i64,str,str,f64,i64,i64,i64,i64
20,"""Money Train""","""Action""",2.871458,589922,589922,4306,1995
20,"""Money Train""","""Comedy""",2.871458,589922,589922,4306,1995
20,"""Money Train""","""Crime""",2.871458,589922,589922,4306,1995
20,"""Money Train""","""Drama""",2.871458,589922,589922,4306,1995
20,"""Money Train""","""Thriller""",2.871458,589922,589922,4306,1995


Considering the purpose of the project... decide if we need to delete the movies without values

In [33]:
#drop a linhas com null values
#df_clean = df_exploded.drop_nulls()

### **3.1.2. Exploration, visualization and conclusions blablabla?**

## **4. Exploratory Data Analysis 2**

#### **4.1. Dataset for this exploration**

#### **4.2. Data Preparation**

##### 4.2.1. Ratings time

According with dataset source, the Timestamps represent seconds since midnight Coordinated Universal Time (UTC) of January 1, 1970.


In [52]:
# Convert the 'timestamp' column to datetime and create a new column 'date'
df_ratings = df_ratings.with_columns(
    (pl.col("timestamp") * 1000).cast(pl.Datetime("ms")).alias("datetime")
)

# The * by 1000 is because the original timestamp is in seconds, and we need milliseconds for polars

df_ratings.head()

userId,movieId,rating,timestamp,datetime
i64,i64,f64,i64,datetime[ms]
1,17,4.0,944249077,1999-12-03 19:24:37
1,25,1.0,944250228,1999-12-03 19:43:48
1,29,2.0,943230976,1999-11-22 00:36:16
1,30,5.0,944249077,1999-12-03 19:24:37
1,32,5.0,943228858,1999-11-22 00:00:58


##### 4.2.3. Tags time

In [53]:
df_tags.head()

userId,movieId,tag,timestamp
i64,i64,str,i64
22,26479,"""Kevin Kline""",1583038886
22,79592,"""misogyny""",1581476297
22,247150,"""acrophobia""",1622483469
34,2174,"""music""",1249808064
34,2174,"""weird""",1249808102


In [54]:
# Convert the 'timestamp' column to datetime and create a new column 'date'
df_tags= df_tags.with_columns(
    (pl.col("timestamp") * 1000).cast(pl.Datetime("ms")).alias("datetime")
)

# The * by 1000 is because the original timestamp is in seconds, and we need milliseconds for polars

df_tags.head()

userId,movieId,tag,timestamp,datetime
i64,i64,str,i64,datetime[ms]
22,26479,"""Kevin Kline""",1583038886,2020-03-01 05:01:26
22,79592,"""misogyny""",1581476297,2020-02-12 02:58:17
22,247150,"""acrophobia""",1622483469,2021-05-31 17:51:09
34,2174,"""music""",1249808064,2009-08-09 08:54:24
34,2174,"""weird""",1249808102,2009-08-09 08:55:02


## **4. Visualization and Anaysis**

Gráficos:
	•	Scatter n_ratings vs avg_rating (com tendência)
	•	Boxplots de avg_rating por género
	•	Importância das features no modelo
	•	Insights:
	•	Géneros com maiores médias
	•	Géneros mais populares
	•	Relação entre popularidade e qualidade percebida


## **5. Conclusions**

	•	Identificar que fatores explicam melhor as boas avaliações.
	•	Propor extensão:
	•	Modelo preditivo por utilizador (colaborativo)
	•	Integração com tags para enriquecer o conteúdo
